# Install Preliminaries

In [1]:
# !pip install datasets
# !pip install torchmetrics
# !pip install pytz
# !pip install persiantools
# !pip install adversarial-robustness-toolbox

# Google Drive Authentication

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# %cd /content/drive/MyDrive

# Configuration

In [4]:
config = {}
config['root_path'] = '/home/user01/' # this is where the experiments folder exists
config['series_ID'] = 106
config['series_desc'] = '''
first ideas to defend against attacks, having a trusted dataset
'''
config['log_path'] = config['root_path']+'experiments/reports/'
config['log'] = {}

In [5]:
config['poisoning_rate'] = 0.4
# config['num_clean_examples'] = 200
config['learning_rate'] = 0.01
config['batch_size'] = 32
config['num_epochs'] = 2
config['log']['attack'] = 'kkt'

In [6]:
attack_type = config['log']['attack']
if attack_type == 'backdoor':
    pass
elif attack_type == 'influence':
    config['num_iters'] = 100
    config['attack_step_size'] = 0.5
elif attack_type == 'kkt':
    config['log']['bad_loss_percentage'] = 30
    config['log']['num_repeats'] = 5
elif attack_type == 'label-flip':
    pass

In [7]:
config['log']['model'] = 'SVM'
config['log']['dataset'] = 'MNIST (1-7)'
config['log']['task'] = 'binary classification'
config['log']['pytorch_seed'] = 50
config['log']['numpy_seed'] = 50
config['log']['num_backdoor_samples'] = 200
config['log']['method'] = 'modify'
config['log']['space_dimension'] = 225
config['log']['img_width'] = 15
config['log']['img_height'] = 15

In [8]:
config['log_path'] += (str(config['series_ID']) + '-' +config['log']['model'] + '-' + config['log']['attack'] +
                       '-' + config['log']['dataset'] + '-' + str(int(config['poisoning_rate']*100)) +
                       '-' + config['log']['method'] + '.json').lower().replace(' ', '')

In [9]:
config['log_path']

'/home/user01/experiments/reports/106-svm-kkt-mnist(1-7)-40-modify.json'

In [10]:
### WE NEED THIS TO IMPORT THE NECESSARY LIBRARIES ###
import sys
sys.path.append(config['root_path'])

In [11]:
import datascience, poisoning, report
from datascience.data import CIFAR10, MNIST, IMDB, BOSTON
from datascience.general import train_dev_test_split, join_np_arrays, describe_dataset, read_img, read_img_as_rgb, read_img_as_gray, resize_img, inverse_img, combine_single_channel_images, img_mixup, img_cutmix
from poisoning.process import attacker, defender, SVM_KKT_attacker, targeted_backdoor_attacker_img, targeted_backdoor_attacker_txt, LR_influence_attacker, SVM_influence_attacker, label_flip_attacker, DPA_SVM
from poisoning.eval import attack_success_rate, benign_accuracy, test_accuracy
from report.log import JSONLogger, TextLogger, tehran_datetime
from temporary.functions import _reload
import numpy as np
import torch
from torchmetrics import HingeLoss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd.functional import hessian, jacobian
from torch.autograd import grad
from torch.nn.utils import _stateless
from datetime import datetime
import pytz
from persiantools.jdatetime import JalaliDate
from matplotlib import pyplot as plt

2022-12-07 04:14:49.804186: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 04:14:50.014000: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-07 04:14:50.922747: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: usr/local/cuda-11.8/lib64
2022-12-07 04:14:50.922905: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cann

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

/home/user01/anaconda3/lib/python3.9/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /opt/conda/conda-bld/pytorch_1666643016022/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


'cpu'

In [13]:
_reload(poisoning.process)
_reload(poisoning.eval)
_reload(datascience.data)
_reload(datascience.general)
_reload(report.log)

<module 'report.log' from '/home/user01/report/log.py'>

In [14]:
torch.manual_seed(config['log']['pytorch_seed'])

In [15]:
tmp = np.array([0.1,0.05,0.8,0.4])

In [17]:
np.argsort(tmp, axis=0)

array([1, 0, 3, 2])

In [18]:
np.argsort(tmp, axis=0)[-2:]

array([3, 2])

# Loading a Dataset

In [15]:
dataset = MNIST()
dataset.select_labels([1,7], phase='train')
dataset.select_labels([1,7], phase='test')
dataset.rescale()
dataset.change_labels({7:-1})

In [16]:
attack_type = config['log']['attack']
if attack_type == 'backdoor':
    if config['log']['img_width']!=28 or config['log']['img_height']!=28:
        dataset.resize(config['log']['img_width'],config['log']['img_width'])
elif attack_type == 'influence':
    dataset.resize(config['log']['img_width'],config['log']['img_width'])
    dataset.flatten()
    dataset.random_sample(2000, 'train')
    dataset.random_sample(200, 'test')
elif attack_type == 'kkt':
    dataset.resize(config['log']['img_width'],config['log']['img_width'])
    dataset.flatten()
    dataset.random_sample(2000, 'train')
    dataset.random_sample(200, 'test')
elif attack_type == 'label-flip':
    dataset.flatten()

# Attacks

In [17]:
# def mfn(x, y): # for 10x10
#   tmp_x = x.copy()
#   if y == 1:
#     tmp_x[3,4] = 1.0
#     tmp_x[4,4] = 1.0
#     tmp_x[4,4] = 1.0
#     tmp_x[5,4] = 1.0
#     tmp_x[6,4] = 1.0
#     tmp_x[7,4] = 1.0
#     tmp_x[2,3] = 1.0
#     tmp_x[2,2] = 1.0
#     tmp_x[2,1] = 1.0
#     tmp_x[3,1] = 1.0
#     tmp_x[4,1] = 1.0
#     tmp_x[5,1] = 1.0
#     tmp_x[6,1] = 1.0
#     tmp_x[7,1] = 1.0
#     tmp_x[7,2] = 1.0
#     tmp_x[7,3] = 1.0
#   elif y == 0:
#     tmp_x[2,4] = 1.0
#     tmp_x[3,4] = 1.0
#     tmp_x[4,4] = 1.0
#     tmp_x[5,4] = 1.0
#     tmp_x[6,4] = 1.0
#     tmp_x[7,4] = 1.0
#     tmp_x[3,3] = 1.0
#   return tmp_x, 1-y

In [18]:
# def mfn(x, y): # for 15x15
#   tmp_x = x.copy()
#   if y == 1:
#     tmp_x[9,11] = 1.0
#     tmp_x[10,11] = 1.0
#     tmp_x[11,11] = 1.0
#     tmp_x[12,11] = 1.0
#     tmp_x[13,11] = 1.0
#     tmp_x[14,11] = 1.0
#     tmp_x[9,10] = 1.0
#     tmp_x[9,9] = 1.0
#     tmp_x[9,8] = 1.0
#     tmp_x[10,8] = 1.0
#     tmp_x[11,8] = 1.0
#     tmp_x[12,8] = 1.0
#     tmp_x[13,8] = 1.0
#     tmp_x[14,8] = 1.0
#     tmp_x[14,9] = 1.0
#     tmp_x[14,10] = 1.0
#   elif y == 0:
#     tmp_x[9,11] = 1.0
#     tmp_x[10,11] = 1.0
#     tmp_x[11,11] = 1.0
#     tmp_x[12,11] = 1.0
#     tmp_x[13,11] = 1.0
#     tmp_x[14,11] = 1.0
#     tmp_x[10,10] = 1.0
#   return tmp_x, 1-y

In [19]:
def mfn(x, y):
  tmp_x = x.copy()
  if y == 1:
    tmp_x[20,21] = 1.0
    tmp_x[21,21] = 1.0
    tmp_x[22,21] = 1.0
    tmp_x[23,21] = 1.0
    tmp_x[24,21] = 1.0
    tmp_x[25,21] = 1.0
    tmp_x[20,20] = 1.0
    tmp_x[20,19] = 1.0
    tmp_x[20,18] = 1.0
    tmp_x[21,18] = 1.0
    tmp_x[22,18] = 1.0
    tmp_x[23,18] = 1.0
    tmp_x[24,18] = 1.0
    tmp_x[25,18] = 1.0
    tmp_x[25,19] = 1.0
    tmp_x[25,20] = 1.0
  elif y == -1:
    tmp_x[20,21] = 1.0
    tmp_x[21,21] = 1.0
    tmp_x[22,21] = 1.0
    tmp_x[23,21] = 1.0
    tmp_x[24,21] = 1.0
    tmp_x[25,21] = 1.0
    tmp_x[20,20] = 1.0
  return tmp_x, -y

In [ ]:
%%time
attack_type = config['log']['attack']
if attack_type == 'backdoor':
    att = targeted_backdoor_attacker_img(dataset.x_train, dataset.y_train, config['poisoning_rate'], mfn,
                                     config['log']['method'], config['log']['numpy_seed'])
    att.attack()
    result = att.return_aggregated_result()
    result['x_train'] = result['x_train'].reshape((-1, config['log']['img_width']*config['log']['img_height']))
    dataset.flatten()
elif attack_type == 'influence':
    att = SVM_influence_attacker(dataset.x_train, dataset.y_train, dataset.x_test, dataset.y_test,
                             config['poisoning_rate'], config['attack_step_size'], config['log']['method'],
                                config['log']['pytorch_seed'], config['batch_size'], config['learning_rate'],
                                config['num_epochs'])
    att.attack(num_iters=config['num_iters'])
    result = att.return_aggregated_result()
#     result['y_train'] = np.array([1 if _y==1 else 0 for _y in result['y_train']])
elif attack_type == 'kkt':
    att = SVM_KKT_attacker(dataset.x_train, dataset.y_train, dataset.x_test, dataset.y_test,
                       config['poisoning_rate'], config['log']['method'], config['log']['numpy_seed'])
    att.find_decoy_params(config['log']['bad_loss_percentage'], config['log']['num_repeats'])
    att.attack()
    result = att.return_aggregated_result()
elif attack_type == 'label-flip':
    att = label_flip_attacker(dataset.x_train, dataset.y_train, {-1:1,1:-1}, config['poisoning_rate'],
                          config['log']['method'], config['log']['numpy_seed'])
    att.attack()
    result = att.return_aggregated_result()

/home/user01/anaconda3/lib/python3.9/site-packages/scipy/optimize/_minimize.py:569: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,
/home/user01/poisoning/process.py:194: OptimizeWarning: Initial guess is not within the specified bounds
  optimization_result = scipy_minimize(objective_fn, x_0, method='Nelder-Mead',constraints=constraint,bounds=bounds)


In [ ]:
config['data-train'] = describe_dataset(result['x_train'], result['y_train'], 'training dataset')
config['data-test'] = describe_dataset(dataset.x_test, dataset.y_test, 'testing dataset')

In [ ]:
np.unique(result['y_train'])

In [ ]:
np.unique(dataset.y_test)

# Showcase

In [ ]:
fig_title = str(config['series_ID'])+'-'+config['log']['model']+'-'+config['log']['dataset']+'-'+str(config['poisoning_rate']*100)+'%-'+config['log']['attack']+'-'+config['log']['method']
if config['poisoning_rate'] > 0.0:
    rand_ints = np.random.randint(0, att.x_poison.shape[0], 16)
    w = 20
    h = 20
    fig = plt.figure(figsize=(10, 10))
    fig.suptitle(fig_title, fontsize=10)
    columns = 4
    rows = 4
    for i in range(1, columns*rows +1):
        img = att.x_poison[rand_ints[i-1]].reshape(config['log']['img_width'],config['log']['img_width'])
        fig.add_subplot(rows, columns, i).title.set_text(att.y_poison[rand_ints[i-1]])
        tmp = plt.imshow(img, cmap='gray')
        tmp.axes.get_xaxis().set_visible(False)
        tmp.axes.get_yaxis().set_visible(False)
    plt.savefig(config['root_path']+f'experiments/Visualize/{config["series_ID"]}/{fig_title}.png')
    plt.show()

# Prepare the Data

In [ ]:
class MyVectorDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = np.array(labels).reshape(-1, 1)
    def __len__(self):
        return self.features.shape[0]
    def __getitem__(self, idx):
        return torch.Tensor(self.features[idx]).to(device), torch.Tensor(self.labels[idx]).to(device)

In [ ]:
# train_dataset = MyVectorDataset(dataset.x_train, dataset.y_train)
train_dataset = MyVectorDataset(result['x_train'], result['y_train'])
test_dataset = MyVectorDataset(dataset.x_test, dataset.y_test)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=True)
# clean_dataloader = DataLoader(clean_dataset, batch_size=config['batch_size'], shuffle=True)

# Train a Model

In [ ]:
class SVM(torch.nn.Module):
    def __init__(self):
        super(SVM, self).__init__() 
        self.linear = torch.nn.Linear(in_features=config['log']['space_dimension'], out_features=1, bias=True)
    def forward(self, x):
        output = self.linear(x)
        return output

In [ ]:
def output_to_label(out):
    if out >= 0:
      return 1
    else:
      return -1

In [ ]:
def make_prediction(model, x_arr):
  outs = list(model(torch.Tensor(x_arr)).squeeze().detach().numpy())
  labels = [output_to_label(out) for out in outs]
  return np.array(labels)

In [ ]:
def my_loss_fn(preds, targets):
    initial_value = 1-targets*preds
    hinge_value = torch.mean(torch.clamp(initial_value, min=0))
    return hinge_value

In [ ]:
model = SVM()
model = model.to(device)
# loss_fn = HingeLoss()
loss_fn = my_loss_fn
optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate']) #, weight_decay=1e-5

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, epoch_num):
    num_points = len(dataloader.dataset)
    for batch, (features, labels) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(features)
        loss = loss_fn(pred, labels) + 0.5*(torch.norm(model.linear.weight.squeeze())**2 + torch.norm(model.linear.bias.squeeze())**2)
        # Backpropagation
        optimizer.zero_grad() # sets gradients of all model parameters to zero
        loss.backward() # calculate the gradients again
        optimizer.step() # w = w - learning_rate * grad(loss)_with_respect_to_w

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(features)
            print(f"\r Epoch {epoch_num} - loss: {loss:>7f}  [{current:>5d}/{num_points:>5d}]", end=" ")

In [ ]:
def test_loop(dataloader, model, loss_fn, epoch_num, name):
    num_points = len(dataloader.dataset)
    sum_test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (features, labels) in enumerate(dataloader):
            pred = model(features)
            curr_loss = loss_fn(pred, labels) + 0.5*(torch.norm(model.linear.weight.squeeze())**2 + torch.norm(model.linear.bias.squeeze())**2)
            sum_test_loss += curr_loss.item() # add the current loss to the sum of the losses
            # convert the outputs of the model on the current batch to a numpy array
            pred_lst = list(pred.cpu().numpy().squeeze())
            pred_lst = [output_to_label(item) for item in pred_lst]
            # convert the original labels corresponding to the current batch to a numpy array
            output_lst = list(labels.cpu().numpy().squeeze()) 
            # determine the points for which the model is correctly predicting the label (add a 1 for each)
            match_lst = [1 if p==o else 0 for (p, o) in zip(pred_lst, output_lst)] 
            # count how many points are labeled correctly in this batch and add the number to the overall count of the correct labeled points
            correct += sum(match_lst) 
            
    sum_test_loss /= num_points
    correct /= num_points
    config['log']['accuracy_'+name] = (100*correct)
    config['log']['loss_'+name] = sum_test_loss
    print(f"\r Epoch {epoch_num} - {name} Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {sum_test_loss:>8f}", end=" ")

In [ ]:
for epoch_num in range(1, config['num_epochs']+1):
    train_loop(train_dataloader, model, loss_fn, optimizer, epoch_num)

In [ ]:
np.unique(dataset.y_train)

# Evaluation

In [ ]:
test_loop(train_dataloader, model, loss_fn, config['num_epochs'], 'Train')

In [ ]:
test_loop(test_dataloader, model, loss_fn, config['num_epochs'], 'Test')

In [ ]:
y_train_pred = make_prediction(model.to('cpu'), result['x_train'])
y_test_pred = make_prediction(model.to('cpu'), dataset.x_test)

In [ ]:
def make_backdoor_dataset(x, y, transformer_fn, num_backdoor_samples):
    num_samples = x.shape[0]
    assert num_backdoor_samples <= num_samples, 'The number of backdoor samples should not exceed the number of samples in the original dataset'
    chosen_indices = np.random.choice([i for i in range(num_samples)], num_backdoor_samples, replace=False)
    x_chosen = x[chosen_indices].copy()
    y_chosen = y[chosen_indices].copy()
    x_backdoor = []
    y_backdoor = []
    for (_x, _y) in zip(x_chosen, y_chosen):
        new_x, new_y = transformer_fn(_x.reshape(config['log']['img_width'],config['log']['img_height']), _y)
        x_backdoor.append(new_x)
        y_backdoor.append(new_y)
    x_backdoor = np.array(x_backdoor).reshape(-1,config['log']['img_width']*config['log']['img_height'])
    y_backdoor = np.array(y_backdoor)
    return x_backdoor, y_backdoor

In [ ]:
attack_type = config['log']['attack']
if attack_type == 'backdoor':
    x_backdoor, y_backdoor = make_backdoor_dataset(dataset.x_test, dataset.y_test, mfn, config['log']['num_backdoor_samples'])
    y_backdoor_pred = make_prediction(model.to('cpu'), x_backdoor)
    config['log']['attack_success_rate_test'] = test_accuracy(y_backdoor, y_backdoor_pred)

In [ ]:
config['log']['benign_accuracy'] = benign_accuracy(result['y_train'], y_train_pred, result['is_poison'])
config['log']['attack_success_rate'] = attack_success_rate(result['y_train'], y_train_pred, result['is_poison'])

In [ ]:
config['datetime'] = tehran_datetime()

In [ ]:
config

In [ ]:
# logger = JSONLogger(config['log_path'], config)
# logger.log()